In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# from numpy.random import multivariate_normal,permutation
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
from pandas import DataFrame, Series

np.random.seed(20160614)
tf.set_random_seed(20160614)

%config IPCompleter.greedy=True
%matplotlib inline
%matplotlib notebook

In [ ]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [ ]:
num_units1 = 2
num_units2 = 2

x = tf.placeholder(tf.float32, [None, 2])

w1 = tf.Variable(tf.truncated_normal([2, num_units1]))
b1 = tf.Variable(tf.zeros([num_units1]))
hidden1 = tf.nn.tanh(tf.matmul(x, w1)+b1)

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
hidden2 = tf.nn.tanh(tf.matmul(hidden1, w2) + b2)

w0 = tf.Variable(tf.zeros([num_units,10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.sigmoid(tf.matmul(hidden2, w0) + b0)

In [ ]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.arg_max(p, 1), tf.arg_max(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.groval_variables_initializer())

In [ ]:
i=0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss,accuracy],feed_dict={x:mnist.test.images,t:mnist.test.labels})
        print ('Step:%d, Loss:%f, Accuracy:%f' % (i, loss_val, acc_val) )

In [ ]:
images, labels = mnist.test.images,mnist.test.labels
p_val = sess.run(p, feed_dict={x:images, t:labels})
fig = plt.figure(figsize = (8,15))

for i in range(10):
    c = 1
    for (image, label, pred) in zip(images, labels, p_val):
        prediction, actual = np.argmax(pred), np.argmax(label)
#         print(i)
        if prediction != i:
            continue
        if (c < 4 and i == actual) or (c >= 4 and i != actual):
            subplot = fig.add_subplot(10,6,i*6+c)
            subplot.set_xticks([])
            subplot.set_yticks([])
            subplot.set_title('%d/%d' % (prediction,actual))
            subplot.imshow(image.reshape((28,28)), vmin=0, vmax=1,cmap=plt.cm.gray_r, interpolation="nearest")
            c += 1
            if c > 6:
                break